# cnn text classification

In [6]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
 

from tensorflow import keras
from tensorflow.python.keras.datasets import imdb
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing import sequence


from tensorboard import summary as summary_lib

#전처리 lib
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.corpus import stopwords

import re

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.4.0


# 전처리에 대한 자세한 내용은
  1. Kaggle API 연동 (TF_tutorial에 있는)
  2. 현 연구원님의 자료 (Kaggle에 있는 data_pipeline EDA)

In [8]:
default_path='~/.kaggle/competitions/word2vec-nlp-tutorial/'
train = pd.read_csv(default_path+"labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [9]:
#매번 위와 같은 작업을 반복할 수 없으니 함수로 만들어봅니다.
def review_to_words( raw_review ):
    # 1. HTML 태그 지우기
    review_text = BeautifulSoup(raw_review, "html5lib").get_text()
    
    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들을 나누기
    words = letters_only.lower().split()

    # 4. 파이썬에서 리스트보다 set에서의 찾기가 더 빠르다고 합니다.
    stops = set(stopwords.words("english"))
    
    # 5. 불용어가 아닌 것만 남기기
    meaningful_words = [w for w in words if not w in stops]
    
    # 6. 불용어가 아닌 텍스트를 공백 단위로 나누어서 리턴
    return( " ".join( meaningful_words ))

#함수를 만들었으니 본격적으로 사용해봅시다.
#전체 데이터의 라인 수를 얻습니다.
num_reviews = train["review"].size

#보통 엄청 크고 아름다운 PC를 가졌다면 위처럼 실행하면 됩니다.
#아니라면....컴퓨터가 일을 제대로 하고 있는 지 확인이 필요할 겁니다.
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in range( 0, num_reviews ):
    #1000라인마다 잘하고 있는 지 보고서를 제출하게 합니다.
    if( (i+1)%1000 == 0 ):
        print ("Review %d of %d\n" % ( i+1, num_reviews ))
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [10]:
#tf내의 내장된 keras기능을 사용하여, 문장을 전처리를 진행하고, 단어 vocab을 dictionary화 한다.

t = Tokenizer()
t.fit_on_texts(clean_train_reviews)
variables = t.texts_to_sequences(clean_train_reviews)
labels = np.array(train['sentiment'])

from sklearn.model_selection import train_test_split

x_train_variable, x_test_variable, y_train, y_test = train_test_split(
            variables, labels, test_size=0.2, random_state=10)

In [14]:
vocab_size = 80000
sentence_size = 50
embedding_size = 50
# model_dir = tempfile.mkdtemp()
model_dir = './checkpoint/cnn_classifier'


# we assign the first indices in the vocabulary to special tokens that we use
# for padding, as start token, and for indicating unknown words
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2

print("Pad sequences (samples x time)")
x_train = sequence.pad_sequences(x_train_variable, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                                maxlen=sentence_size,
                                truncating='post',
                                padding='post', 
                                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)

word_index = t.word_index
word_inverted_index = {v + index_offset: k for k, v in word_index.items()}

# The first indexes in the map are reserved to represent things other than tokens
word_inverted_index[pad_id] = '<PAD>'
word_inverted_index[start_id] = '<START>'
word_inverted_index[oov_id] = '<OOV>'

for i in range(0, 10):
    print(i, word_inverted_index[i])
  
def index_to_text(indexes):
    return ' '.join([word_inverted_index[i] for i in indexes])

print(index_to_text(variables[0]))

Pad sequences (samples x time)
x_train shape: (20000, 50)
x_test shape: (5000, 50)
0 <PAD>
1 <START>
2 <OOV>
3 movie
4 film
5 one
6 like
7 good
8 time
9 even
stories look age korda happy lonely done back manages taking together fanshawe together knoxville dvd us well form jungle music enough would hilarious uwe dvd made less oscar aliens painful knoxville find testament find actual <OOV> tell look story title constant scary touching monkey korda fight c brosnan bad james giving redeeming much im context solid although wanted fashion begin nazi film korda type look feature makes women thought turned korda kazakos aymeric script <START> korda based time end first based maybe would idea working actual <OOV> whole dialogue car far kristel thrills carry local killing lordi tension mukhsin casting mysterious extreme close korda help much sequence korda halleck cartoons baloo killing lordi think journeymen turn get best frewer redeeming gave dyer dvd rick korda done surprised hilarious quite 

In [15]:
#input function

#각 문장의 길이를 계산한다, max 길이는 200

x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])

def parser(x, length, y):
    features = {"x": x, "len": length}
    return features, y

#len을 활용하여 기존 전처리 이후의 길이를 보존
#from_tensor_slices를 활용하면 numpy 데이터 구조에서 쉽게 변환


def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train_variable))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    dataset = dataset.repeat()
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((x_test, x_len_test, y_test))
    dataset = dataset.batch(100)
    dataset = dataset.map(parser)
    iterator = dataset.make_one_shot_iterator()
    
    return iterator.get_next()

# CNN Classification with tf-keras

CNN을 활용하여 text를 분류해보자, n-gram의 효과로 활용

https://www.semanticscholar.org/paper/Learning-to-Rank-Short-Text-Pairs-with-Deep-Neural-Severyn-Moschitti/452f7411af7d471dd3ba84c2b06b2aaffc38cdb9

Embedding Layer -> Dropout -> Conv1D -> GlobalMax1D -> Hidden Dense Layer -> Dropout -> Output Layer

In [20]:
#keras로 간단한 모델 구성

max_words = 50

# Build the model
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape=(max_words,), name="posts"))
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#convert model keras to estimator
model_dir = os.path.join(os.getcwd(), "checkpoint")
os.makedirs(model_dir, exist_ok=True)
print("model_dir ", model_dir)
estiamtor_model = keras.estimator.model_to_estimator(keras_model=model, model_dir=model_dir)

model_dir  /Users/user/git/DeepNLP/TF_Tutorial/checkpoint
INFO:tensorflow:Using the Keras model from memory.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/user/git/DeepNLP/TF_Tutorial/checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12160c2b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
def input_fn(features, labels=None, shuffle=False):
    
    input_val = tf.estimator.inputs.numpy_input_fn(
        x={"posts_input": features},
        y=labels,
        shuffle=shuffle)
    return input_val
    
    
#     dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
#     dataset = dataset.shuffle(buffer_size=len(x_train_variable))
#     dataset = dataset.batch(100)
#     dataset = dataset.map(parser)
#     dataset = dataset.repeat()
#     iterator = dataset.make_one_shot_iterator()
    
#     return iterator.get_next()
input_fn

<function __main__.input_fn>

In [23]:
estiamtor_model.train(input_fn=input_fn(x_train, labels, shuffle=True))

ValueError: Length of tensors in x and y is mismatched. All elements in x and y must have the same length.
Shapes in x: {'posts_input': (20000, 50), '__target_key__': (25000,)}
Shape for y: (25000,)


In [78]:
all_classifiers = {}

def train_and_evaluate(classifier):
    # 예측 테스트를 위해 모델을 학습시키고 저장한다.
    all_classifiers[classifier.model_dir] = classifier
    classifier.train(input_fn=train_input_fn, steps=1)
    eval_results = classifier.evaluate(input_fn=eval_input_fn)
    predictions = np.array([p['logistic'][0] for p in classifier.predict(input_fn=eval_input_fn)])
    
    # name scopes의 재사용을 위해 graph를 reset한다.
    tf.reset_default_graph()
    
    pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool),
                             num_thresholds=21)
    
    with tf.Session() as sess:
        writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
        writer.add_summary(sess.run(pr), global_step=0)
        writer.close()

In [79]:
#head: pre-made estimator로 평가를 할 때, 일정한 함수를 사용하게 세팅
head = tf.contrib.estimator.binary_classification_head()

def cnn_model_fn(features, labels, mode, params):
    #embedding layer를 선언한다.
    input_layer = tf.contrib.layers.embed_sequence(
                    features['x'],
                    vocab_size,
                    embedding_size,
                    initializer=params['embedding_initializer']
                    )

    training = (mode == tf.estimator.ModeKeys.TRAIN)
    dropout_emb = tf.layers.dropout(inputs=input_layer,
                                   rate=0.2,
                                   training=training)

    conv = tf.layers.conv1d(
            inputs=dropout_emb,
            filters=32,
            kernel_size=3,
            padding='same',
            activation=tf.nn.relu)
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)  
    dropout_hidden = tf.layers.dropout(inputs=hidden, rate=0.2, training=training)
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    #prediction 진행 시, None
    if labels is not None:
        labels = tf.reshape(labels, [-1, 1])
    
    optimizer = tf.train.AdamOptimizer() #여러가지 Optimizer 활용가능
    
    def _train_op_fn(loss):
#         tf.summary('loss', loss)
        return optimizer.minimize(
                loss=loss,
                global_step=tf.train.get_global_step())

    
    return head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits,
        train_op_fn=_train_op_fn)

params = {'embedding_initializer': tf.random_uniform_initializer(-1.0, 1.0)}
cnn_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                        model_dir=os.path.join(model_dir, 'cnn'),
                                        params=params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoint/cnn_classifier/cnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13239a828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [80]:
#학습 후, 결과치를 tensorboard로 확인
# tensorboard --logdir=./checkpoint/cnn_classifier/
train_and_evaluate(cnn_classifier)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/cnn_classifier/cnn/model.ckpt-101000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 101001 into ./checkpoint/cnn_classifier/cnn/model.ckpt.
INFO:tensorflow:loss = 8.498208e-06, step = 101001
INFO:tensorflow:Loss for final step: 8.498208e-06.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-07-11:37:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/cnn_classifier/cnn/model.ckpt-101001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-07-11:37:56
INFO:tensorflow:Saving dict for global step 101001: accuracy = 0.8706, 

In [89]:
# 직접 prediction으로 테스트 해 본다

def text_to_index(sentence):
    # Remove punctuation characters except for the apostrophe
    translator = str.maketrans('', '', string.punctuation.replace("'", ''))
    tokens = sentence.translate(translator).lower().split()
    return np.array([1] + [word_index[t] if t in word_index else oov_id for t in tokens])

def print_predictions(sentences):
    indexes = [text_to_index(sentence) for sentence in sentences]
    x = sequence.pad_sequences(indexes, 
                               maxlen=sentence_size, 
                               truncating='post',
                               padding='post',
                               value=pad_id)
    length = np.array([min(len(x), sentence_size) for x in indexes])
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": x, "len": length}, shuffle=False)
    predictions = {}
    for path, classifier in all_classifiers.items():
        predictions[path] = [p['logistic'][0] for p in classifier.predict(input_fn=predict_input_fn)]
    for idx, sentence in enumerate(sentences):
        print(sentence)
        for path in all_classifiers:
            print("\t{} {}".format(path, predictions[path][idx]))
#             predictions[path][idx]
    
    return predictions[path][idx]

In [90]:
print_predictions([
    'I really liked the movie!',
    'Hated every second of it...'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/cnn_classifier/cnn/model.ckpt-101001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I really liked the movie!
	./checkpoint/cnn_classifier/cnn 1.0
Hated every second of it...
	./checkpoint/cnn_classifier/cnn 1.4289345926954411e-05


1.4289346e-05

In [91]:
print_predictions(['fuck you', 'this movie sucks'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/cnn_classifier/cnn/model.ckpt-101001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
fuck you
	./checkpoint/cnn_classifier/cnn 0.1322735995054245
this movie sucks
	./checkpoint/cnn_classifier/cnn 2.3199970829426566e-19


2.319997e-19

지금까지 했던 것을 모두 활용하여 제출용 데이터를 만들어봅시다.

In [92]:
#테스트 데이터를 읽어봅시다.
test = pd.read_csv(default_path+"testData.tsv", header=0, delimiter="\t", quoting=3 )

#테스트 파일은 이렇게 생겼다고 합니다.
print (test.head())

#이 파일은 "sentiment" 행이 없습니다.
print (test.shape)

#불용어 제거도하고 태그들도 지우고 난후 데이터를 저장할 장소를 만들어둡니다.
num_reviews = len(test["review"])
clean_test_reviews = []

print ("Cleaning and parsing the test set movie reviews...\n")
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

           id                                             review
0  "12311_10"  "Naturally in a film who's main themes are of ...
1    "8348_2"  "This movie is a disaster within a disaster fi...
2    "5828_4"  "All in all, this is a movie for kids. We saw ...
3    "7186_2"  "Afraid of the Dark left me with the impressio...
4   "12128_7"  "A very accurate depiction of small time mob l...
(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Revi

In [98]:
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.

sentimental = []

for i in range(len(clean_test_reviews)):
    if ( (i+1) % 1000 == 0):
        print ("Current Progress %d \n" % (i+1))
    sentimental.append(print_predictions([clean_test_reviews[i]]))

NameError: name 'asda' is not defined

In [99]:
#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":sentimental} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv( "Bag_of_Words_model_test.csv", index=False, quoting=3 )

ValueError: array length 0 does not match index length 25000

In [114]:
#0.5 기준으로 값들을 변환

def correct_val(x):
    if x >= 0.5:
        x = 1
    else:
        x = 0
    
    return x

final_result = output['sentiment'].apply(correct_val)

In [115]:
#알아보기 쉽게 데이터랑 붙여두는 편이 좋을 거 같습니다.
output = pd.DataFrame( data={"id":test["id"], "sentiment":final_result} )

#지금까지 처리한 결과를 파일로 저장합니다.
output.to_csv( "final_bof.csv", index=False, quoting=3 )

#최종 점수는 0.8344가 나왔습니다. 단순한 cnn 1 layer 모델 기준. 향후 파인 튜닝 하면 더 증가 할 가능성이 있ㄴ에ㅛ